In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
%matplotlib inline

In [106]:
#一个函数用于序列转换，一个函数用于检测“N”碱基
def long_short(data):
    seq_list = []
    for i in data[0]:
        seq_list.append(str(i)[480:521])
    return np.array(seq_list)

def check_N(data1,data2):
    seq_list = []
    gene_list = []
    for i in range(len(data1)):
        if str(data1[i]).find("N")<0:
            seq_list.append(data1[i])
            gene_list.append(data2[i])
    return np.array(seq_list),np.array(gene_list)

def np_score(data):
    sum_s = 0.0
    pos = 0.0
    for i in range(len(data)):
        if "P" in data[i][1]:
            pos+=data[i][-1]
        sum_s += data[i][-1]
    if sum_s != 0:
        return pos/sum_s
    else:
        return 0.5
    
def train_np_score(data,name):
    sum_s = 0.0
    pos = 0.0
    for i in range(len(data)):
        if name != data[i][1]:
            if "P" in data[i][1]:
                pos+=data[i][-1]
            sum_s += data[i][-1]
    if sum_s != 0:
        return pos/sum_s
    else:
        return 0.5
    
def seq_fasta(data,label,file_dir):
    with open(file_dir, 'w') as f:
        for i in range(len(data)):
            if label[i] == 1:
                f.write('>P' + str(i + 1) + '\n')
            else:
                f.write('>N' + str(i + 1) + '\n')
            f.write(data[i] + '\n')

In [107]:
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]

In [108]:
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        trainPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        trainNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        testPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        testNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        trainPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        trainNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        testPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        testNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        
        #获取文件信息
        trainPos_seq = pd.read_csv(trainPosSeq_file,header=None)
        trainNeg_seq = pd.read_csv(trainNegSeq_file,header=None)
        testPos_seq = pd.read_csv(testPosSeq_file,header=None)
        testNeg_seq = pd.read_csv(testNegSeq_file,header=None)
        trainPos_gene = pd.read_csv(trainPosGene_file) 
        trainNeg_gene = pd.read_csv(trainNegGene_file)
        testPos_gene = pd.read_csv(testPosGene_file)
        testNeg_gene = pd.read_csv(testNegGene_file)
        #序列转41-nt长度
        trainPos_seq = long_short(trainPos_seq)
        trainNeg_seq = long_short(trainNeg_seq)
        testPos_seq = long_short(testPos_seq)
        testNeg_seq = long_short(testNeg_seq)
        #基因转35长度
        trainPos_gene = trainPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainNeg_gene = trainNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testPos_gene = testPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testNeg_gene = testNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainPos_seq,trainPos_gene = check_N(trainPos_seq,np.array(trainPos_gene))
        trainNeg_seq,trainNeg_gene = check_N(trainNeg_seq,np.array(trainNeg_gene))
        testPos_seq,testPos_gene = check_N(testPos_seq,np.array(testPos_gene))
        testNeg_seq,testNeg_gene = check_N(testNeg_seq,np.array(testNeg_gene))
        #基因变Float
        trainPos_gene = np.float32(trainPos_gene)
        trainNeg_gene = np.float32(trainNeg_gene)
        testPos_gene = np.float32(testPos_gene)
        testNeg_gene = np.float32(testNeg_gene)
        #打乱顺序，再取等长
        trainNeg_seq, trainNeg_gene = shuffle(trainNeg_seq,trainNeg_gene, random_state=1)
        testNeg_seq, testNeg_gene = shuffle(testNeg_seq, testNeg_gene, random_state=1)

        trainNeg_seq = trainNeg_seq[:len(trainPos_seq)]
        testNeg_seq = testNeg_seq[:len(testPos_seq)]

        trainPos_label = np.ones(len(trainPos_seq))
        trainNeg_label = np.zeros(len(trainNeg_seq))
        testPos_label = np.ones(len(testPos_seq))
        testNeg_label = np.zeros(len(testNeg_seq))   

#五折交叉存储fasta文件 
#         kf = KFold(5,True,10)
#         train_seq = np.append(trainPos_seq,trainNeg_seq)
#         trainLabel = np.append(trainPos_label,trainNeg_label)
#         for i,[train_index, test_index] in enumerate(kf.split(train_seq)):
#             X_train = train_seq[train_index]
#             X_test = train_seq[test_index]
#             Y_train = trainLabel[train_index]
#             Y_test = trainLabel[test_index]
#             file_dir = "Blastn/"+type_name[typei]+"+"+cell_name[celli]+"/KFold_"+str(i)+"/"
#             if not os.path.exists(file_dir):
#                 os.makedirs(file_dir)
#             seq_fasta(X_train,Y_train,file_dir+"train.fasta")
#             seq_fasta(X_test,Y_test,file_dir+"test.fasta")
        
# 文件存储为fasta的文件格式
#         train_seq = np.append(trainPos_seq,trainNeg_seq)
#         trainLabel = np.append(trainPos_label,trainNeg_label)
#         test_seq = np.append(testPos_seq,testNeg_seq)
#         testLabel = np.append(testPos_label,testNeg_label)
#         file_dir = "Blastn/"+type_name[typei]+"+"+cell_name[celli]+"/"
#         if not os.path.exists(file_dir):
#             os.makedirs(file_dir)
#         seq_fasta(train_seq,trainLabel,file_dir+"train.fasta")
#         seq_fasta(test_seq,testLabel,file_dir+"test.fasta")

#训练数据获取结果
        train_seq = np.append(trainPos_seq,trainNeg_seq)
        trainLabel = np.append(trainPos_label,trainNeg_label)
        re_dir = "Blastn/"+type_name[typei]+"+"+cell_name[celli]+"/result_train.csv"
        data = pd.read_csv(re_dir,header=None)
        train_score = []
        for i in range(len(trainPos_seq)):
            train_score.append(train_np_score(np.array(data.loc[data[0] == "P"+str(i+1)]),"P"+str(i+1) ))
        for i in range(len(trainPos_seq)):
            train_score.append(train_np_score(np.array(data.loc[data[0] == "N"+str(i+1+len(trainPos_seq))]),"N"+str(i+1+len(trainPos_seq))))
        pd.DataFrame(np.append(np.array(train_score).reshape(-1,1),trainLabel.reshape(-1,1),axis=1)).to_csv("Blastn/"+type_name[typei]+"_"+cell_name[celli]+"_result_train.csv" ,header=None)

#测试数据获取结果
#         test_seq = np.append(testPos_seq,testNeg_seq)
#         testLabel = np.append(testPos_label,testNeg_label)
#         re_dir = "Blastn/"+type_name[typei]+"+"+cell_name[celli]+"/result.csv" 
#         data = np.array(pd.read_csv(re_dir,header=None)) 
#         data = pd.read_csv(re_dir,header=None)
#         test_score = []
#         for i in range(len(testPos_seq)):
#             test_score.append(np_score(np.array(data.loc[data[0] == "P"+str(i+1)])))
#         for i in range(len(testPos_seq)):
#             test_score.append(np_score(np.array(data.loc[data[0] == "N"+str(i+1+len(testPos_seq))])))
#         pd.DataFrame(np.append(np.array(test_score).reshape(-1,1),testLabel.reshape(-1,1),axis=1)).to_csv("Blastn/"+type_name[typei]+"_"+cell_name[celli]+"_result.csv" ,header=None)

In [20]:
# 文件存储为fasta的文件格式
# train_seq = np.append(trainPos_seq,trainNeg_seq)
# trainLabel = np.append(trainPos_label,trainNeg_label)
# file_dir = "Blastn/"+type_name[typei]+"+"+cell_name[celli]+"/"
# if not os.path.exists(file_dir):
#     os.makedirs(file_dir)
# seq_fasta(train_seq,trainLabel,file_dir+"train.fasta")
# test_seq = np.append(testPos_seq,testNeg_seq)
# testLabel = np.append(testPos_label,testNeg_label)
# seq_fasta(test_seq,testLabel,file_dir+"test.fasta")

In [43]:
re_dir = "Blastn/"+type_name[typei]+"+"+cell_name[celli]+"/result.csv" 
data = np.array(pd.read_csv(re_dir,header=None))
data = pd.read_csv(re_dir,header=None)
test_score = []
for i in range(len(testPos_seq)):
    test_score.append(train_np_score(np.array(data.loc[data[0] == "P"+str(i+1)]),"P"+str(i+1) ))
for i in range(len(testPos_seq)):
    test_score.append(np_score(np.array(data.loc[data[0] == "N"+str(i+1+len(testPos_seq))]),"N"+str(i+1+len(testPos_seq))))
pd.DataFrame(np.append(np.array(test_score).reshape(-1,1),testLabel.reshape(-1,1))).to_csv("Blastn/"+type_name[typei]+"_"+cell_name[celli]+"_result.csv" ,header="score")

In [109]:
len(test_score)

17936

In [88]:
Th = []
Acc = []
Mcc = []
Auc = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        file_name = "Blastn/" + type_name[typei] + "_"+cell_name[celli]+"_result.csv"
        y_pred = np.array(pd.read_csv(file_name,header=None,index_col=0).iloc[:,0])
        y_true = np.array(pd.read_csv(file_name,header=None,index_col=0).iloc[:,1])
        th,_,_,_,_,acc,_,_,mcc,auc = metricsCal.get_train_metrics(y_pred,y_true)
        Th.append(th)
        Acc.append(acc)
        Mcc.append(mcc)
        Auc.append(auc)

In [93]:
Acc

[0.7226153576232528,
 0.7892674994456353,
 0.7075584371267701,
 0.7222998602515472,
 0.7404164312807683,
 0.6874863268431416,
 0.6978590754028938,
 0.6546095486244157,
 0.6483854433623782,
 0.7028594281143771,
 0.6194962196256014,
 0.6190900981266726]

In [90]:
#计算12个物种的结果
Acc_test = []
Mcc_test = []
Auc_test = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        file_name = "Blastn/" + type_name[typei] + "_"+cell_name[celli]+"_result.csv"
        y_pred = np.array(pd.read_csv(file_name,header=None,index_col=0).iloc[:,0])
        y_true = np.array(pd.read_csv(file_name,header=None,index_col=0).iloc[:,1])
        _,_,_,_,acc,_,_,mcc,auc = metricsCal.get_test_metrics(y_pred,y_true,0.51)
        Acc_test.append(acc)
        Mcc_test.append(mcc)
        Auc_test.append(auc)

In [96]:
np.mean(Auc_test[0:6]),np.mean(Auc_test[6:12])

(0.746296629722114, 0.6711812724858165)

In [97]:
np.mean(Auc[0:6]),np.mean(Auc[6:12])

(0.746296629722114, 0.6711812724858165)

In [98]:
np.mean(Mcc[0:6]),np.mean(Mcc[6:12])

(0.3711482221459467, 0.2592277860411468)

In [105]:
np.append(np.array(train_score).reshape(-1,1),trainLabel.reshape(-1,1),axis=1).shape

(30312, 2)